In [187]:
### Import Statements ###
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as ms
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.linear_model import LogisticRegressionCV, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from scipy.sparse import coo_matrix, hstack
import scipy
from sklearn.model_selection import GridSearchCV
import re

In [188]:
df = pd.read_csv("train_E6oV3lV.csv")
df2 = pd.read_csv("test_tweets_anuFYb8.csv")
df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [189]:
def remo(features):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",features[0]).split())

In [190]:
df['tweet'] = df[['tweet']].apply(remo,axis=1)
df2['tweet'] = df2[['tweet']].apply(remo,axis=1)

In [191]:
df.head()

,id,label,tweet
0,1,0,when a father is dysfunctional and is so selfi...
1,2,0,thanks for lyft credit i can t use cause they ...
2,3,0,bihday your majesty
3,4,0,model i love u take with u all the time in ur
4,5,0,factsguide society now motivation


In [197]:
params = {"tfidf__ngram_range": [(1, 2)],
          "svc__C": [.01, .1, 1, 10, 100]}

clf = Pipeline([("tfidf", TfidfVectorizer(sublinear_tf=True)),
                ("svc", LinearSVC(loss='hinge')])

gs = GridSearchCV(clf, params, verbose=2, n_jobs=-1)
gs.fit(df['tweet'], df['label'])
print(gs.best_estimator_)
print(gs.best_score_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] svc__C=0.01, tfidf__ngram_range=(1, 2) ..........................
[CV] svc__C=0.01, tfidf__ngram_range=(1, 2) ..........................
[CV] svc__C=0.01, tfidf__ngram_range=(1, 2) ..........................
[CV] svc__C=0.1, tfidf__ngram_range=(1, 2) ...........................
[CV] ........... svc__C=0.01, tfidf__ngram_range=(1, 2), total=   4.9s
[CV] svc__C=0.1, tfidf__ngram_range=(1, 2) ...........................
[CV] ........... svc__C=0.01, tfidf__ngram_range=(1, 2), total=   5.0s
[CV] svc__C=0.1, tfidf__ngram_range=(1, 2) ...........................
[CV] ............ svc__C=0.1, tfidf__ngram_range=(1, 2), total=   5.0s
[CV] svc__C=1, tfidf__ngram_range=(1, 2) .............................
[CV] ........... svc__C=0.01, tfidf__ngram_range=(1, 2), total=   5.3s
[CV] svc__C=1, tfidf__ngram_range=(1, 2) .............................
[CV] ............ svc__C=0.1, tfidf__ngram_range=(1, 2), total=   5.2s
[CV] svc__C=1, tf

[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   37.2s finished


Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
 ...e', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0))])
0.9646142293974094


In [198]:
predicted = gs.predict(df2['tweet'])

In [199]:
predicted

array([0, 0, 0, ..., 0, 0, 0])

In [200]:
final_predict = pd.DataFrame(predicted,columns=['label'])
result = pd.DataFrame(df2['id'],columns=['id'])
result = pd.concat([result,final_predict],axis=1)
result.to_csv('final_predictions2.csv',index=False)

In [201]:
result['label'].value_counts()

0    16209
1      988
Name: label, dtype: int64